In [33]:
import numpy as np
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
root = 'KuaiRec 2.0/'

small_matrix = pd.read_csv(root + "data/small_matrix.csv")

user_features = pd.read_csv(root + "data/user_features.csv")

item_daily_features = pd.read_csv(root + "data/item_daily_features.csv")

caption_category = pd.read_csv(root + "data/kuairec_caption_category_translated.csv", lineterminator='\n')

# Small Matrix
Contains core user-video interaction data with essential metrics for analysis.

## Columns
### Identifiers
- user_id: Unique identifier for each user
- video_id: Unique identifier for each video

### Viewing Metrics
- play_duration: Actual time user spent watching (in seconds)
- video_duration: Total length of video (in seconds)
- watch_ratio: Percentage of video watched (play_duration / video_duration)

### Temporal Data
- time: Time of day when interaction occurred
- date: Calendar date of interaction
- timestamp: Full datetime of interaction

In [3]:
small_matrix

,user_id,video_id,play_duration,video_duration,time,date,timestamp,watch_ratio
0,14,148,4381,6067,2020-07-05 05:27:48.378,20200705.0,1.593898e+09,0.722103
1,14,183,11635,6100,2020-07-05 05:28:00.057,20200705.0,1.593898e+09,1.907377
2,14,3649,22422,10867,2020-07-05 05:29:09.479,20200705.0,1.593898e+09,2.063311
3,14,5262,4479,7908,2020-07-05 05:30:43.285,20200705.0,1.593898e+09,0.566388
4,14,8234,4602,11000,2020-07-05 05:35:43.459,20200705.0,1.593899e+09,0.418364
...,...,...,...,...,...,...,...,...
4676565,7162,2267,11908,5467,NaN,NaN,NaN,2.178160
4676566,7162,2065,11919,6067,NaN,NaN,NaN,1.964562
4676567,7162,1296,16690,19870,NaN,NaN,NaN,0.839960
4676568,7162,4822,11862,24400,NaN,NaN,NaN,0.486148


## Feature Selection
As part of data cleaning, we will only be requiring certain metrics. Refer to the report for more details. <br>

### Steps
1. Column Selection
   - Retains only essential columns: user_id, video_id, time, watch_ratio
   - Removes redundant or unused features

2. Time Standardization
   - Converts 'time' column to datetime format
   - Invalid time values are converted to NaT (Not a Time)

3. Outlier Removal
   - Filters out unrealistic watch ratios by clipping watch_ratio to (<= 5)
   - Ensures data quality for viewing metrics

4. Data Quality Checks
   - Verifies missing values across columns
   - Confirms proper data types

In [4]:
# Columns to keep
cleaned_small_matrix = small_matrix[['user_id', 'video_id', 'time', 'watch_ratio']]

In [5]:
cleaned_small_matrix['time'] = pd.to_datetime(cleaned_small_matrix['time'], errors='coerce')

In [6]:
cleaned_small_matrix = cleaned_small_matrix[cleaned_small_matrix['watch_ratio'] <= 5]

In [7]:
cleaned_small_matrix.isnull().sum()

user_id             0
video_id            0
time           181074
watch_ratio         0
dtype: int64

In [8]:
cleaned_small_matrix.dtypes

user_id                 int64
video_id                int64
time           datetime64[ns]
watch_ratio           float64
dtype: object

In [9]:
cleaned_small_matrix

,user_id,video_id,time,watch_ratio
0,14,148,2020-07-05 05:27:48.378,0.722103
1,14,183,2020-07-05 05:28:00.057,1.907377
2,14,3649,2020-07-05 05:29:09.479,2.063311
3,14,5262,2020-07-05 05:30:43.285,0.566388
4,14,8234,2020-07-05 05:35:43.459,0.418364
...,...,...,...,...
4676565,7162,2267,NaT,2.178160
4676566,7162,2065,NaT,1.964562
4676567,7162,1296,NaT,0.839960
4676568,7162,4822,NaT,0.486148


In [10]:
earliest_date = cleaned_small_matrix['time'].min()
latest_date = cleaned_small_matrix['time'].max()

print(f"Earliest date: {earliest_date}")
print(f"Latest date: {latest_date}")

Earliest date: 2020-07-04 02:23:26.060000
Latest date: 2020-09-05 23:57:23.683000


# Train, Validation, Test Split on Small Matrix

## Overview
Data is split chronologically to maintain temporal integrity and simulate real-world prediction scenarios.

## Split Details
### Training Set (31 days)
- Date Range: July 4, 2020 - August 3, 2020
- Primary dataset for model training
- Represents earliest time period

### Validation Set (16 days)
- Date Range: August 4, 2020 - August 19, 2020
- Used for model tuning and hyperparameter optimization
- Represents intermediate time period

### Test Set (17 days)
- Date Range: August 20, 2020 - September 5, 2020
- Reserved for final model evaluation
- Represents most recent time period

### Other notes
- Non-overlapping date ranges ensure data independence
- Chronological split preserves temporal patterns and trends

In [11]:
# Split the data based on date ranges
train_data = cleaned_small_matrix[
    (cleaned_small_matrix['time'] >= '2020-07-04') &
    (cleaned_small_matrix['time'] < '2020-08-04')  # Use < for non-overlapping
].copy()

val_data = cleaned_small_matrix[
    (cleaned_small_matrix['time'] >= '2020-08-04') &
    (cleaned_small_matrix['time'] < '2020-08-20')  # Use < for non-overlapping
].copy()

test_data = cleaned_small_matrix[
    (cleaned_small_matrix['time'] >= '2020-08-20') &
    (cleaned_small_matrix['time'] <= '2020-09-05')
].copy()

# Optionally, reset indices
train_data.reset_index(drop=True, inplace=True)
val_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

# User Features
We will be selecting several features for their predictive value in user engagement analysis

## Selected Features Overview
This subset focuses on key user characteristics and engagement metrics.

### User Identity & History
- user_id: Unique identifier for each user
- register_days: Number of days since user registration

### Activity Metrics
- user_active_degree: User's overall activity level score
- is_lowactive_period: Boolean flag indicating if user is in a period of low activity (1=yes, 0=no)

### Content Creation Status
- is_live_streamer: Boolean indicating if user conducts live streams (1=yes, 0=no)
- is_video_author: Boolean indicating if user creates video content (1=yes, 0=no)

### Social Network Metrics
- follow_user_num: Number of users this user follows
- fans_user_num: Number of followers/fans this user has
- friend_user_num: Number of mutual connections (both users follow each other)

In [12]:
user_features

,user_id,user_active_degree,is_lowactive_period,is_live_streamer,is_video_author,follow_user_num,follow_user_num_range,fans_user_num,fans_user_num_range,friend_user_num,...,onehot_feat8,onehot_feat9,onehot_feat10,onehot_feat11,onehot_feat12,onehot_feat13,onehot_feat14,onehot_feat15,onehot_feat16,onehot_feat17
0,0,high_active,0,0,0,5,"(0,10]",0,0,0,...,184,6,3,0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,full_active,0,0,0,386,"(250,500]",4,"[1,10)",2,...,186,6,2,0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,full_active,0,0,0,27,"(10,50]",0,0,0,...,51,2,3,0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,full_active,0,0,0,16,"(10,50]",0,0,0,...,251,3,2,0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,full_active,0,0,0,122,"(100,150]",4,"[1,10)",0,...,99,4,2,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7171,7171,full_active,0,0,1,52,"(50,100]",1,"[1,10)",0,...,259,1,4,0,1.0,0.0,0.0,0.0,0.0,0.0
7172,7172,full_active,0,0,0,45,"(10,50]",2,"[1,10)",2,...,11,2,0,0,1.0,0.0,0.0,0.0,0.0,0.0
7173,7173,full_active,0,0,0,615,500+,3,"[1,10)",2,...,51,2,2,0,1.0,0.0,0.0,0.0,0.0,0.0
7174,7174,full_active,0,0,0,959,500+,0,0,0,...,107,3,2,0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# Columns to keep
cleaned_user_features = user_features[['user_id',
    'user_active_degree',
    'is_lowactive_period',
    'is_live_streamer',
    'is_video_author',
    'follow_user_num',
    'fans_user_num',
    'friend_user_num',
    'register_days']]

In [14]:
cleaned_user_features.isnull().sum()

user_id                0
user_active_degree     0
is_lowactive_period    0
is_live_streamer       0
is_video_author        0
follow_user_num        0
fans_user_num          0
friend_user_num        0
register_days          0
dtype: int64

In [15]:
cleaned_user_features

,user_id,user_active_degree,is_lowactive_period,is_live_streamer,is_video_author,follow_user_num,fans_user_num,friend_user_num,register_days
0,0,high_active,0,0,0,5,0,0,107
1,1,full_active,0,0,0,386,4,2,327
2,2,full_active,0,0,0,27,0,0,116
3,3,full_active,0,0,0,16,0,0,105
4,4,full_active,0,0,0,122,4,0,225
...,...,...,...,...,...,...,...,...,...
7171,7171,full_active,0,0,1,52,1,0,283
7172,7172,full_active,0,0,0,45,2,2,109
7173,7173,full_active,0,0,0,615,3,2,167
7174,7174,full_active,0,0,0,959,0,0,241


# Item Daily Features

Aggregates daily video metrics into summary statistics while maintaining temporal separation between train/validation/test sets to prevent data leakage.

## Data Transformation Steps
1. Date Formatting
  - Converts date column from YYYYMMDD string to datetime
  - Invalid dates handled gracefully (converted to NaT)

2. Feature Filtering & Selection
  - Filters by date range and public visibility status
  - Retains key video metadata and engagement metrics:
    * Identifiers: video_id, author_id
    * Content info: video_type, video_tag_name, video_duration
    * Engagement metrics: show_cnt, play_cnt, play_duration
    * Social metrics: like_cnt, comment_cnt, share_cnt, follow_cnt, collect_cnt

3. Aggregation Process
  - Groups by: video_id, author_id, video_type
  - Aggregation methods:
    * Tags: Concatenated, deduplicated, and sorted
    * Duration: Mean value
    * Engagement metrics: Summed over period
  - Missing values handled:
    * Duration: Filled with mean duration
    * Tags: Defaulted to 'Unknown'
    * Collection count: Filled with mean value

In [16]:
item_daily_features

,video_id,date,author_id,video_type,upload_dt,upload_type,visible_status,video_duration,video_width,video_height,...,download_cnt,download_user_num,report_cnt,report_user_num,reduce_similar_cnt,reduce_similar_user_num,collect_cnt,collect_user_num,cancel_collect_cnt,cancel_collect_user_num
0,0,20200705,3309,NORMAL,2020-03-30,ShortImport,public,5966.0,720,1280,...,8,8,0,0,3,3,NaN,NaN,NaN,NaN
1,0,20200706,3309,NORMAL,2020-03-30,ShortImport,public,5966.0,720,1280,...,2,2,0,0,5,5,NaN,NaN,NaN,NaN
2,0,20200707,3309,NORMAL,2020-03-30,ShortImport,public,5966.0,720,1280,...,2,2,0,0,0,0,NaN,NaN,NaN,NaN
3,0,20200708,3309,NORMAL,2020-03-30,ShortImport,public,5966.0,720,1280,...,3,3,0,0,3,3,NaN,NaN,NaN,NaN
4,0,20200709,3309,NORMAL,2020-03-30,ShortImport,public,5966.0,720,1280,...,2,2,2,1,1,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343336,10723,20200905,236,NORMAL,2020-09-05,ShortImport,public,4833.0,720,1280,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
343337,10724,20200905,5271,NORMAL,2020-09-05,LongImport,public,54720.0,720,1280,...,1,1,0,0,0,0,0.0,0.0,0.0,0.0
343338,10725,20200905,1924,NORMAL,2020-09-05,ShortImport,public,15800.0,576,1024,...,5,5,0,0,4,4,0.0,0.0,0.0,0.0
343339,10726,20200905,7604,NORMAL,2020-09-05,ShortImport,public,5132.0,528,960,...,2,2,0,0,1,1,0.0,0.0,0.0,0.0


In [17]:
# Convert 'date' column to datetime
item_daily_features['date'] = pd.to_datetime(
    item_daily_features['date'].astype(str),
    format='%Y%m%d',
    errors='coerce'  # Handles any incorrect formats by setting them as NaT
)

In [18]:
def aggregate_item_features(item_features, start_date, end_date):
    # Filter item_daily_features within the specified date range
    filtered_features = item_features[
        (item_features['date'] >= start_date) &
        (item_features['date'] <= end_date) &
        (item_features['visible_status'] == 'public')  # Ensure visibility
    ].copy()
    
    # Select relevant columns
    filtered_features = filtered_features[[
        'video_id',
        'author_id',
        'video_type',
        'video_tag_name',
        'video_duration',
        'show_cnt',
        'play_cnt',
        'play_duration',
        'like_cnt',
        'comment_cnt',
        'share_cnt',
        'follow_cnt',
        'collect_cnt'
    ]]
    
    # Define aggregation functions
    def concat_tags(x):
        tags = [str(tag) for tag in x if pd.notna(tag)]
        return ', '.join(sorted(set(tags)))
    
    aggregated = filtered_features.groupby(['video_id', 'author_id', 'video_type']).agg({
        'video_tag_name': concat_tags,
        'video_duration': 'mean',
        'show_cnt': 'sum',
        'play_cnt': 'sum',
        'play_duration': 'sum',
        'like_cnt': 'sum',
        'comment_cnt': 'sum',
        'share_cnt': 'sum',
        'follow_cnt': 'sum',
        'collect_cnt': 'sum'
    }).reset_index()
    
    # Handle missing values
    aggregated['video_duration'] = aggregated['video_duration'].fillna(filtered_features['video_duration'].mean())
    aggregated['video_tag_name'] = aggregated['video_tag_name'].fillna('Unknown')
    aggregated['collect_cnt'] = aggregated['collect_cnt'].fillna(filtered_features['collect_cnt'].mean())
    
    return aggregated


In [19]:
# Define date ranges for each split
train_start_date = '2020-07-04'
train_end_date = '2020-08-03'

val_start_date = '2020-08-04'
val_end_date = '2020-08-19'

test_start_date = '2020-08-20'
test_end_date = '2020-09-05'

# Aggregate features for each split
aggregated_train_features = aggregate_item_features(item_daily_features, train_start_date, train_end_date)
aggregated_val_features = aggregate_item_features(item_daily_features, val_start_date, val_end_date)
aggregated_test_features = aggregate_item_features(item_daily_features, test_start_date, test_end_date)

In [20]:
aggregated_train_features.isnull().sum()

video_id          0
author_id         0
video_type        0
video_tag_name    0
video_duration    0
show_cnt          0
play_cnt          0
play_duration     0
like_cnt          0
comment_cnt       0
share_cnt         0
follow_cnt        0
collect_cnt       0
dtype: int64

In [21]:
aggregated_train_features

,video_id,author_id,video_type,video_tag_name,video_duration,show_cnt,play_cnt,play_duration,like_cnt,comment_cnt,share_cnt,follow_cnt,collect_cnt
0,0,3309,NORMAL,建筑,5966.000000,381704,224131,2041770158,14147,199,48,7812,28.0
1,1,4978,NORMAL,,12355.165414,134944,141314,2449672004,8259,59,63,246,0.0
2,2,939,NORMAL,生活,8000.000000,647628,669696,8638941159,3176,30,19,403,0.0
3,3,5889,NORMAL,树木,12355.165414,14435,8906,77964107,939,7,26,37,0.0
4,4,4284,NORMAL,卖衣服,18000.000000,781,603,7389559,3,0,0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5970,10136,3340,NORMAL,穿戴,9966.000000,481,229,1021794,2,0,0,0,0.0
5971,10137,1122,NORMAL,正面,15200.000000,18713,8384,112616176,147,0,0,6,0.0
5972,10138,6358,NORMAL,正面,6066.000000,63,16,81268,4,1,0,0,0.0
5973,10139,4983,NORMAL,室内,7500.000000,2571,1177,9787783,40,1,0,1,0.0


In [22]:
aggregated_val_features.isnull().sum()

video_id          0
author_id         0
video_type        0
video_tag_name    0
video_duration    0
show_cnt          0
play_cnt          0
play_duration     0
like_cnt          0
comment_cnt       0
share_cnt         0
follow_cnt        0
collect_cnt       0
dtype: int64

In [23]:
aggregated_test_features.isnull().sum()

video_id          0
author_id         0
video_type        0
video_tag_name    0
video_duration    0
show_cnt          0
play_cnt          0
play_duration     0
like_cnt          0
comment_cnt       0
share_cnt         0
follow_cnt        0
collect_cnt       0
dtype: int64

# Caption Category

Processes video metadata including captions, categories, and topic tags in both original and English languages. <br>
Before this step, we have already translated the captions and categories from Chinese to English to improve caption/tags/category analysis in feature engineering, modelling and evaluation stages. This is done using the Qwen2.5-14B model

## Cleaning Steps
1. Data Validation & Type Conversion
  - Removes rows with invalid video_ids
  - Converts video_id to int64 format
  - Removes carriage return characters from column names

2. Feature Selection
  - Retains essential metadata columns:
    * Identifiers: video_id
    * Text content: manual_cover_text, caption
    * Categorization: topic tags, hierarchical categories
    * English translations of all text fields

3. Missing Value Handling
  - Text fields (captions, categories): Empty string ('')
  - Topic tags: Empty list representation ('[]')\

## Selected Features
### Original Language
- manual_cover_text: Cover text for video
- caption: Video caption/description
- topic_tag: Topic-related tags
- Category hierarchy:
 * first_level_category_name
 * second_level_category_name
 * third_level_category_name

### English Translations
- english_caption
- english_topic_tag
- English category hierarchy:
 * english_first_level_category_name
 * english_second_level_category_name
 * english_third_level_category_name

In [24]:
caption_category

,video_id,manual_cover_text,caption,topic_tag,first_level_category_id,first_level_category_name,second_level_category_id,second_level_category_name,third_level_category_id,third_level_category_name,english_caption,english_first_level_category_name,english_second_level_category_name,english_third_level_category_name,english_topic_tag
0,0,UNKNOWN,精神小伙路难走 程哥你狗粮慢点撒,[],8.0,颜值,673.0,颜值随拍,-124.0,UNKNOWN,"The spirit of the young man is tough;程哥, pleas...",Beauty index,Snap of good looks,UNKNOWN,[]
1,1,UNKNOWN,NaN,[],27.0,高新数码,-124.0,UNKNOWN,-124.0,UNKNOWN,man,HighTech Digital,UNKNOWN,UNKNOWN,[]
2,2,UNKNOWN,晚饭后，运动一下！,[],9.0,喜剧,727.0,搞笑互动,-124.0,UNKNOWN,"After dinner, get some exercise!",Comedy,Humorous interaction,UNKNOWN,[]
3,3,UNKNOWN,我平淡无奇，惊艳不了时光，温柔不了岁月，我只想漫无目的的走走，努力发笔小财，给自己买花 自己长大.,[],26.0,摄影,686.0,主题摄影,2434.0,景物摄影,"I am unremarkable, unable to stun time or soft...",Photography,Theme Photography,Landscape photography,[]
4,4,五爱街最美美女 一天1q,#搞笑 #感谢快手我要上热门 #五爱市场 这真是完美搭配啊！,"[五爱市场,感谢快手我要上热门,搞笑]",5.0,时尚,737.0,营销售卖,2596.0,女装,#Humorous #Thanks快准热浪 #Wulai Market This is a ...,Fashion,Sales promotion,Women's clothing,"[Wulai Market, thanks for Fastly Me Wanting to..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10727,10723,UNKNOWN,昨天爱你，今天爱你，明天也爱你，丫头，别担心，我以后都会爱你，我的小傻瓜@公主没烦恼 、(O...,[],33.0,自拍,-124.0,UNKNOWN,-124.0,UNKNOWN,"Love you yesterday, love you today, and love y...",Selfie,UNKNOWN,UNKNOWN,[]
10728,10724,UNKNOWN,#感谢推广小助手 #感谢快手绿色平台 #,"[感谢快手绿色平台,感谢推广小助手]",6.0,明星娱乐,-124.0,UNKNOWN,-124.0,UNKNOWN,#ThanksPromotionAssistant #ThanksFastlyGreenP...,Stars' Entertainment,UNKNOWN,UNKNOWN,Thanks for the Green Platform of Kuaishou and ...
10729,10725,UNKNOWN,NaN,[],15.0,艺术,170.0,表演,-124.0,UNKNOWN,man,Art,Performance,UNKNOWN,[]
10730,10726,老人言,老人言，喜欢留个关注加红心 #老人言 @今天拍点啥(O840386039) @快手活动中...,[老人言],38.0,读书,696.0,文学赏析,2477.0,民间俗语,"The old man said, like to leave a follow and a...",Reading,Literary Appreciation,Proverb,[Older People Say]


In [25]:
caption_category = caption_category[
    caption_category['video_id'].notna() & 
    pd.to_numeric(caption_category['video_id'], errors='coerce').notna()
]
caption_category['video_id'] = caption_category['video_id'].astype('int64')

In [26]:
# Clean column names using str.replace
caption_category.columns = caption_category.columns.str.replace('\r', '')

In [27]:
cleaned_caption_category = caption_category[['video_id', 'manual_cover_text', 'caption', 'topic_tag',
       'first_level_category_name', 'second_level_category_name', 'third_level_category_name',
       'english_caption', 'english_first_level_category_name',
       'english_second_level_category_name',
       'english_third_level_category_name', 'english_topic_tag']]
columns_to_fill = [
    'caption',
    'first_level_category_name',
    'second_level_category_name',
    'third_level_category_name',
    'english_first_level_category_name',
    'english_second_level_category_name',
    'english_third_level_category_name'
]

cleaned_caption_category[columns_to_fill] = cleaned_caption_category[columns_to_fill].fillna('')
cleaned_caption_category['topic_tag'].fillna('[]', inplace=True)
# Replace 'man' with empty string for english_caption
cleaned_caption_category['english_caption'] = cleaned_caption_category['english_caption'].replace('man', '', regex=False)

In [28]:
cleaned_caption_category.isnull().sum()

video_id                              0
manual_cover_text                     0
caption                               0
topic_tag                             0
first_level_category_name             0
second_level_category_name            0
third_level_category_name             0
english_caption                       0
english_first_level_category_name     0
english_second_level_category_name    0
english_third_level_category_name     0
english_topic_tag                     0
dtype: int64

In [29]:
cleaned_caption_category

,video_id,manual_cover_text,caption,topic_tag,first_level_category_name,second_level_category_name,third_level_category_name,english_caption,english_first_level_category_name,english_second_level_category_name,english_third_level_category_name,english_topic_tag
0,0,UNKNOWN,精神小伙路难走 程哥你狗粮慢点撒,[],颜值,颜值随拍,UNKNOWN,"The spirit of the young man is tough;程哥, pleas...",Beauty index,Snap of good looks,UNKNOWN,[]
1,1,UNKNOWN,,[],高新数码,UNKNOWN,UNKNOWN,,HighTech Digital,UNKNOWN,UNKNOWN,[]
2,2,UNKNOWN,晚饭后，运动一下！,[],喜剧,搞笑互动,UNKNOWN,"After dinner, get some exercise!",Comedy,Humorous interaction,UNKNOWN,[]
3,3,UNKNOWN,我平淡无奇，惊艳不了时光，温柔不了岁月，我只想漫无目的的走走，努力发笔小财，给自己买花 自己长大.,[],摄影,主题摄影,景物摄影,"I am unremarkable, unable to stun time or soft...",Photography,Theme Photography,Landscape photography,[]
4,4,五爱街最美美女 一天1q,#搞笑 #感谢快手我要上热门 #五爱市场 这真是完美搭配啊！,"[五爱市场,感谢快手我要上热门,搞笑]",时尚,营销售卖,女装,#Humorous #Thanks快准热浪 #Wulai Market This is a ...,Fashion,Sales promotion,Women's clothing,"[Wulai Market, thanks for Fastly Me Wanting to..."
...,...,...,...,...,...,...,...,...,...,...,...,...
10727,10723,UNKNOWN,昨天爱你，今天爱你，明天也爱你，丫头，别担心，我以后都会爱你，我的小傻瓜@公主没烦恼 、(O...,[],自拍,UNKNOWN,UNKNOWN,"Love you yesterday, love you today, and love y...",Selfie,UNKNOWN,UNKNOWN,[]
10728,10724,UNKNOWN,#感谢推广小助手 #感谢快手绿色平台 #,"[感谢快手绿色平台,感谢推广小助手]",明星娱乐,UNKNOWN,UNKNOWN,#ThanksPromotionAssistant #ThanksFastlyGreenP...,Stars' Entertainment,UNKNOWN,UNKNOWN,Thanks for the Green Platform of Kuaishou and ...
10729,10725,UNKNOWN,,[],艺术,表演,UNKNOWN,,Art,Performance,UNKNOWN,[]
10730,10726,老人言,老人言，喜欢留个关注加红心 #老人言 @今天拍点啥(O840386039) @快手活动中...,[老人言],读书,文学赏析,民间俗语,"The old man said, like to leave a follow and a...",Reading,Literary Appreciation,Proverb,[Older People Say]


# Merge Aggregated Features with Each Split

Combines cleaned and preprocessed features from multiple sources into comprehensive datasets
for training, validation, and testing. Uses left joins to preserve all user-video interactions.

## Data Sources
1. Core Interaction Data
  - train_data/val_data/test_data: Base interaction datasets with temporal splits
  - Contains: user_id, video_id, time, watch_ratio

2. User Features
  - cleaned_user_features
  - Contains: Activity metrics, content creation flags, social network stats

3. Video Features
  - aggregated_[train/val/test]_features
  - Contains: Engagement metrics, video metadata, aggregated by time period

4. Content Metadata/Caption Category
  - cleaned_caption_category
  - Contains: Video descriptions, categories, topic tags in multiple languages

## Additional Key Notes
- user_id: Links interaction data with user features
- video_id: Links interaction data with video features and content metadata
- Separate merges for each time period to prevent data leakage
- Each dataset (train/val/test) maintains temporal independence to prevent any form of data leakage
- Video features specific to each time period used in respective merges
- Finally, all cleaned data are exported to the data_exports directory

In [30]:
print(cleaned_small_matrix.columns)
print(cleaned_user_features.columns)
print(aggregated_train_features.columns)
print(cleaned_caption_category.columns)

Index(['user_id', 'video_id', 'time', 'watch_ratio'], dtype='object')
Index(['user_id', 'user_active_degree', 'is_lowactive_period',
       'is_live_streamer', 'is_video_author', 'follow_user_num',
       'fans_user_num', 'friend_user_num', 'register_days'],
      dtype='object')
Index(['video_id', 'author_id', 'video_type', 'video_tag_name',
       'video_duration', 'show_cnt', 'play_cnt', 'play_duration', 'like_cnt',
       'comment_cnt', 'share_cnt', 'follow_cnt', 'collect_cnt'],
      dtype='object')
Index(['video_id', 'manual_cover_text', 'caption', 'topic_tag',
       'first_level_category_name', 'second_level_category_name',
       'third_level_category_name', 'english_caption',
       'english_first_level_category_name',
       'english_second_level_category_name',
       'english_third_level_category_name', 'english_topic_tag'],
      dtype='object')


In [31]:
# Merge Train Data with User Features
joined_train_data = train_data.merge(
    cleaned_user_features,
    on='user_id',
    how='left'
)

# Merge with Aggregated Train Item Features
joined_train_data = joined_train_data.merge(
    aggregated_train_features,
    on='video_id',
    how='left'
)

# Merge with Caption Category Features (assuming caption_category is already cleaned appropriately)
joined_train_data = joined_train_data.merge(
    cleaned_caption_category,
    on='video_id',
    how='left'
)

# Repeat the process for Validation and Test sets
# Merge Validation Data with User Features
joined_val_data = val_data.merge(
    cleaned_user_features,
    on='user_id',
    how='left'
)

# Merge with Aggregated Validation Item Features
joined_val_data = joined_val_data.merge(
    aggregated_val_features,
    on='video_id',
    how='left'
)

# Merge with Caption Category Features
joined_val_data = joined_val_data.merge(
    cleaned_caption_category,
    on='video_id',
    how='left'
)

# Merge Test Data with User Features
joined_test_data = test_data.merge(
    cleaned_user_features,
    on='user_id',
    how='left'
)

# Merge with Aggregated Test Item Features
joined_test_data = joined_test_data.merge(
    aggregated_test_features,
    on='video_id',
    how='left'
)

# Merge with Caption Category Features
joined_test_data = joined_test_data.merge(
    cleaned_caption_category,
    on='video_id',
    how='left'
)


In [34]:
# Create export directory if it doesn't exist
if not os.path.exists(root + 'data_exports'):
    os.makedirs(root + 'data_exports')

# Export all dataframes to CSV
joined_train_data.to_csv(root + 'data_exports/joined_train_data.csv', index=False)
joined_val_data.to_csv(root + 'data_exports/joined_val_data.csv', index=False)
joined_test_data.to_csv(root + 'data_exports/joined_test_data.csv', index=False)
cleaned_small_matrix.to_csv(root + 'data_exports/cleaned_small_matrix.csv', index=False)
aggregated_train_features.to_csv(root + 'data_exports/aggregated_train_features.csv', index=False)
aggregated_val_features.to_csv(root + 'data_exports/aggregated_val_features.csv', index=False)
aggregated_test_features.to_csv(root + 'data_exports/aggregated_test_features.csv', index=False)
cleaned_user_features.to_csv(root + 'data_exports/cleaned_user_features.csv', index=False)
cleaned_caption_category.to_csv(root + 'data_exports/cleaned_caption_category.csv', index=False)

print("All files have been exported successfully!")

All files have been exported successfully!


In [35]:
joined_train_data.shape

(2552082, 35)

In [36]:
joined_val_data.shape

(1376299, 35)

In [37]:
joined_test_data.shape

(541410, 35)